In [1]:
# step-1 importing libraries...
from py2neo import Graph
import pandas as pd
import neo4j
from copy import deepcopy

In [2]:
#step-2 Connecting to neo4j DBMS name'MyDBMS'...
graph = Graph("bolt://localhost:7687",password = 'azeem0786')

In [36]:
#step-3 Return the nodes along with the properties in python, created in neo4j and assigned to variable 'node'...
node=graph.run('match (n) return n.name as name,n.formula as formula;').data()

In [3]:
def BaseGraph(mongo=None):
    baseGraph=graph.run("match (n) \
                        return n.name as name,n.formula as formula,\
                        n.value as value,\
                        labels(n) as Section").to_data_frame()
    return baseGraph

In [4]:
df = BaseGraph()

In [5]:
df

,name,formula,value,Section
0,Rate,None,None,[Price]
1,Quantity,None,None,[Number]
2,Amount,Rate*Quantity,None,[Total]
3,Discount Per,None,None,[Cost]
4,Discount Amount,Discount Per*Amount,None,[Cost]
5,Net Amount,Amount-Discount Amount,None,[Cost]


In [25]:
dernodes = [x for x,y in zip(df['name'],df['formula']) if y != None]
inpnodes = [x for x,y in zip(df['name'],df['formula']) if y == None]

In [26]:
dernodes

['Amount', 'Discount Amount', 'Net Amount']

In [27]:
inpnodes

['Rate', 'Quantity', 'Discount Per']

In [28]:
def FillProp(graph,prop):
    "Fills those nodes which doesnt contain the passed property with None"
    for node in list(graph.nodes()):
        for p in prop:
            if p not in graph.nodes[node].keys():
                graph.nodes[node][p]=None
    return graph;

In [29]:
def DeriveTo(graph,Nodes):
    "returns the nodes to which the list of nodes passed is used for derivation"
    derives=[]
    for l in Nodes:
        try:
            adj=list(graph.adj[l])
        except KeyError:
            print("no adjacent nodes:",l)
            continue
        for i in adj:
            if pd.isnull(graph[l][i]["derive"])==False:
                derives.append(i) 
    return derives;
    
def SeqDerived(graph):
    "Order the Derived nodes based on its relationship"
    From=[x for x,y,v in graph.edges.data("derive") if pd.isnull(v)==False]
    To=[y for x,y,v in graph.edges.data("derive") if pd.isnull(v)==False]
    Input=[x for x in From if x not in To] 
    
    derList=deepcopy(Input)
    while len(Input)!=0:
        Input=DeriveTo(graph=graph,Nodes=Input)
        derList+=Input 
    
    drvOrdered=[derList[x] for x in range(len(derList)) if derList[x] not in derList[x+1:] and pd.isnull(graph.nodes[derList[x]]["DERIVE"])==False ]
    return drvOrdered;

In [40]:
gLease = FillProp(graph=graph,prop=['LINKED'])

TypeError: 'NodeMatcher' object is not callable

In [9]:
#step-4 Extracting the names from the node... 
nodeName={x['name']:None for x in node}

In [10]:
nodeName

{'Rate': None,
 'Quantity': None,
 'Amount': None,
 'Discount Per': None,
 'Discount Amount': None,
 'Net Amount': None}

In [11]:
#step-5 Mapping name of the node with formula of the node... 
nodeProperty={x['name']:x['formula'] for x in node}

In [12]:
nodeProperty

{'Rate': None,
 'Quantity': None,
 'Amount': 'Rate*Quantity',
 'Discount Per': None,
 'Discount Amount': 'Discount Per*Amount',
 'Net Amount': 'Amount-Discount Amount'}

In [13]:
#step-6 Separating input and output nodes...
inpNodes=[x for x,y in nodeProperty.items() if y ==None]
derNodes=[x for x,y in nodeProperty.items() if y !=None]

In [10]:
result = []
for i in inpNodes:
    nodeName[i] = float(input("enter the " +i+ ":"))
        
for d in derNodes:
    formula = nodeProperty[d] 
    for i in inpNodes:
        formula = formula.replace(i,"nodeName['%s']"%i)
    print(formula)
    nodeName[d] = eval(formula)
    result.append(nodeName[d])
    break
for d in derNodes[1:]:
    formula = nodeProperty[d]
    for i in inpNodes[-1:]:
        formula = formula.replace(i,"nodeName['%s']"%i)
    for j in derNodes[:1]:
        formula = formula.replace(j,"nodeName['%s']"%j)
    print(formula)
    nodeName[d] = eval(formula)
    result.append(nodeName[d])
    break
for d in derNodes[2:]:
    formula = nodeProperty[d]
    for j in derNodes:
        nodeName[d] = result[0]-result[1]
        break
    print(formula)

enter the Rate:56
enter the Quantity:58
enter the Discount Per:0.6
nodeName['Rate']*nodeName['Quantity']
nodeName['Discount Per']*nodeName['Amount']
Amount-Discount Amount


In [11]:
nodeName

{'Rate': 56.0,
 'Quantity': 58.0,
 'Amount': 3248.0,
 'Discount Per': 0.6,
 'Discount Amount': 1948.8,
 'Net Amount': 1299.2}